In [4]:
import zeep
import re
import os
import json
import nltk
from nltk.corpus import stopwords


def request_lemmatized(text):

    wsdl = 'http://nlptools.info.uaic.ro/WebPosRo/PosTaggerRoWS?wsdl'
    client = zeep.Client(wsdl=wsdl)
    response = ''
    response += client.service.parseSentence_XML(text)
    response += '🤩'
    return response


def format_text(text):
    text = request_lemmatized(text)
    arr = text.split('\n')
    arr_lemmas = []
    final_tokenized = []
    index = 0
    for line in arr:
        found = re.search('.*(LEMMA="[A-Za-zăâîșț1-9]*").*', line)
        if found:
            res = found.group(1)
            arr_lemmas.append(res)
        found = re.search('🤩', line)
        if found:
            arr_lemmas.append('🤩')
    for lemma in arr_lemmas:
        found = re.search('LEMMA="([A-Za-zăâîșț1-9]*)".*', lemma)
        if found:
            res = found.group(1)
            final_tokenized.append(res)
            index = index + 1
        if index == len(arr_lemmas)-1:
            found = re.search('🤩', line)
            if found:
                final_tokenized.append('🤩')
            break
    return final_tokenized[:-1]


def filter_stopwords(article,stopwerdz):

    with open('stopwords.txt', mode='r') as stopwerds:
        count = 0
        while 1:
            werd = stopwerds.readline()
            werd = werd.strip('\n')
            stopwerdz.append(werd)
            count = count + 1
            if count > 433:
                break
        stw = stopwords.words('romanian')
        stw.extend(stopwerdz)
        filtered_article = []
        for word in article:
            filtered_word =''
            for letter in word:
                if (letter.isalpha()):
                    filtered_word += letter
                else:
                    if(letter == '~'):
                        filtered_word += ' '
            if (filtered_word not in stw and filtered_word != '' and len(filtered_word) >= 2):
                filtered_article.append(filtered_word)
        return filtered_article

def convert_to_string(words):
    return ' '.join(words)

def pipeline(text):
    stopwerdz = []

    sentences = format_text(text)
    filtered_article = filter_stopwords(sentences,stopwerdz)
    filtered_article_string = convert_to_string(filtered_article)

    print(filtered_article_string)

    # return?

if __name__ == '__main__':

    text = "Raluca Turcan susține că există în prezent inechități majore în ceea ce privește salarizarea. Ministrul Muncii a explicat la Digi24 cum un consilier superior dintr-o primărie locală are un salariu mai mare decât unul dintr-un minister, această situație nefiind una izolată. Turcan a dat și exemplul unui angajat dintr-un Consiliu Județean care câștigă cu 2.000 de lei mai mult lunar decât un altul cu aceeași pregătire și încadrare, aflat în alt județ. "
    pipeline(text)

raluca turcan susține exista inechitate major salarizare ministru muncă explica Digi consilier superior dintru primărie local salariu decât dintru minister situație izolat turcan exemplu angajat dintru consiliu județean câștiga leu lunar decât același pregătire încadrare aflat județ
